## ✈️ *TravelTide* - Customer Segmentation Project
---
### Data Understanding and Preparation - SQL Exploration

> ⚠️ **Execution Notice – Notebook 02**
>
> This notebook performs **large-scale SQL exploration and validation**
> on the raw TravelTide database using **Spark + JDBC**,
> including tables with more than **5 million records** (e.g. `sessions`).
>
> The full exploratory SQL workflow is **designed to run in Databricks**
> and is **not intended to be executed end-to-end in Colab**.
>
> **Important:**  
> When running this notebook outside Databricks (e.g. in Colab),
> **do not run the entire notebook**.
>
> Only the two cells under the section  
> **“Session-Level Dataset Materialization”**
> are expected to be executed.
>
> All other SQL cells serve a **documentation and validation purpose**:
> they record the analytical reasoning used to understand the data model,
> verify lifecycle logic, and justify cohort and filtering decisions.
>
> The materialized session-level dataset produced in this step
> is reused by all downstream notebooks (**Notebook 03 onward**).


### **Session-Level Dataset Materialization**

> 🔹 **Execution Scope**  
> The following two cells represent the **only executable steps**
> required to reproduce this notebook outside Databricks.
> They materialize the session-level dataset used by all downstream analyses.


In [7]:
# Install required database and ORM libraries
# psycopg2-binary: PostgreSQL driver
# sqlalchemy: database connection and query interface
!pip -q install psycopg2-binary sqlalchemy

# Core data handling
import pandas as pd

# Database connection utilities
from sqlalchemy import create_engine

# File system and path management
from pathlib import Path
import os

# Mount Google Drive to persist data and outputs
from google.colab import drive
drive.mount('/content/drive')


# Project directory setup
# -------------------------------------------------------------------

# Root project directory on Google Drive
PROJECT_DIR = Path("/content/drive/MyDrive/Masterschool/TravelTide")

# Raw data directory (created if it does not exist)
(DATA_RAW := PROJECT_DIR / "data" / "raw").mkdir(parents=True, exist_ok=True)


# Database connection parameters
# -------------------------------------------------------------------

# Public PostgreSQL database
DB_HOST = "ep-noisy-flower-846766.us-east-2.aws.neon.tech"
DB_NAME = "TravelTide"
DB_USER = "Test"
DB_PW   = "bQNxVzJL4g6u"

# Create SQLAlchemy engine for PostgreSQL
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PW}@{DB_HOST}/{DB_NAME}"
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
query = """
WITH
  -- 1. Sessions after cutoff data
  filtered_sessions AS (
    SELECT *
    FROM sessions
    WHERE session_start > '2023-01-04'
  ),

  -- 2. Users with sufficient activity
  filtered_users AS (
    SELECT
      user_id,
      COUNT(*) AS session_count
    FROM filtered_sessions
    GROUP BY user_id
    HAVING COUNT(*) > 7
  ),

  -- 3. Session-level table for filtered users
  session_level AS (
    SELECT
      -- SESSION FIELDS -------------
      s.session_id,
      s.user_id,
      s.trip_id,
      s.session_start,
      s.session_end,
      s.page_clicks,
      s.flight_discount,
      s.flight_discount_amount,
      s.flight_booked,
      s.hotel_discount,
      s.hotel_discount_amount,
      s.hotel_booked,
      s.cancellation,

      -- USERS FIELDS -------------
      u.birthdate,
      u.gender,
      u.married,
      u.has_children,
      u.home_country,
      u.home_city,
      u.home_airport,
      u.home_airport_lat,
      u.home_airport_lon,
      u.sign_up_date,

      -- FLIGHT FIELDS -------------
      f.origin_airport,
      f.destination,
      f.destination_airport,
      f.seats,
      f.return_flight_booked,
      f.departure_time,
      f.return_time,
      f.checked_bags,
      f.trip_airline,
      f.destination_airport_lat,
      f.destination_airport_lon,
      f.base_fare_usd,

      -- HOTEL FIELDS -------------
      h.hotel_name,
      h.nights,
      h.rooms,
      h.check_in_time,
      h.check_out_time,
      h.hotel_per_room_usd

    FROM filtered_sessions s
    INNER JOIN users u ON s.user_id = u.user_id
    LEFT JOIN flights f ON s.trip_id = f.trip_id
    LEFT JOIN hotels h ON s.trip_id = h.trip_id
    WHERE s.user_id IN (SELECT user_id FROM filtered_users)
  ),

  -- 4. Cancellation sessions inside the window
  cancellations AS (
    SELECT *
    FROM session_level
    WHERE cancellation = TRUE
  ),

  -- 5. Retrieve booking sessions BEFORE cutoff for trips cancelled AFTER cutoff
  sessions_trip_booked_before AS (
    SELECT *
    FROM sessions
    WHERE trip_id IN (
      SELECT trip_id
      FROM cancellations
    )
    AND session_start <= '2023-01-04'  -- booking occurred before the cutoff date
  ),

  -- 6.  Build session-level rows for these earlier booking sessions
  session_level_before AS (
    SELECT
      -- SESSION FIELDS -------------
      s.session_id,
      s.user_id,
      s.trip_id,
      s.session_start,
      s.session_end,
      s.page_clicks,
      s.flight_discount,
      s.flight_discount_amount,
      s.flight_booked,
      s.hotel_discount,
      s.hotel_discount_amount,
      s.hotel_booked,
      s.cancellation,

      -- USER FIELDS -------------
      u.birthdate,
      u.gender,
      u.married,
      u.has_children,
      u.home_country,
      u.home_city,
      u.home_airport,
      u.home_airport_lat,
      u.home_airport_lon,
      u.sign_up_date,

      -- FLIGHT FIELDS -------------
      f.origin_airport,
      f.destination,
      f.destination_airport,
      f.seats,
      f.return_flight_booked,
      f.departure_time,
      f.return_time,
      f.checked_bags,
      f.trip_airline,
      f.destination_airport_lat,
      f.destination_airport_lon,
      f.base_fare_usd,

      -- HOTEL FIELDS -------------
      h.hotel_name,
      h.nights,
      h.rooms,
      h.check_in_time,
      h.check_out_time,
      h.hotel_per_room_usd

    FROM sessions_trip_booked_before s
    INNER JOIN users u ON s.user_id = u.user_id
    LEFT JOIN flights f ON s.trip_id = f.trip_id
    LEFT JOIN hotels h ON s.trip_id = h.trip_id
    WHERE s.user_id IN (SELECT user_id FROM filtered_users)
  ),

  -- 7. Combine both parts
  final_tab AS (
    SELECT * FROM session_level
    UNION ALL
    SELECT * FROM session_level_before
  )

SELECT *
FROM final_tab;
"""

# Output path for the session-level dataset
out_path = DATA_RAW / "session_level_raw.csv"

# Read the SQL query result in chunks to avoid memory issues
# (the session table contains several million records)
chunks = []

for chunk in pd.read_sql_query(query, engine, chunksize=200_000):
    chunks.append(chunk)                 # Accumulate chunks in memory
    print("Loaded chunk:", len(chunk))   # Progress log for long-running queries

# Concatenate all chunks into a single DataFrame
sl_df = pd.concat(chunks, ignore_index=True)

# Export the consolidated session-level dataset to CSV
sl_df.to_csv(out_path, index=False)

Loaded chunk: 49218


### 1. Introduction

This notebook corresponds to the **Data Understanding** phase and the early stages of **Data Preparation** within the CRISP-DM framework.

It focuses on understanding the TravelTide data model directly at the database level using SQL.

The goal is twofold:
- Explore the schema, relationships, and granularity of the data
- Construct a clean and semantically correct session-level raw dataset that will serve as the foundation for all subsequent analysis.

No statistical EDA is performed at this stage. Instead, this notebook ensures that joins, filters, and lifecycle logic are correct before any feature engineering or modeling is attempted.


**Author**: Maria Petralia  
**Project**: TravelTide - Customer Segmentation & Perk Strategy  
**Context**: MasterSchool - Data Science Program  
**Date**: Feb 2026

> **Execution note**  
> This notebook performs direct JDBC reads and exploratory SQL queries on large raw tables.  
> In particular, the `sessions` table contains ~5.4M records.
>
> As a result, **initial execution may take up to 10 minutes**, even with partitioned reads and fetch-size tuning.  
> This is expected behavior and reflects the intentional choice to validate data integrity and lifecycle logic at full granularity before downstream aggregation.

### 2. Load the tables with Spark

#### 2.1 Import Libraries

In [ ]:
# import libraries
import pandas as pd

#### 2.2 Configure PostgreSQL Connection

In [ ]:
# PostgreSQL connection URL
traveltide_url = "jdbc:postgresql://ep-noisy-flower-846766.us-east-2.aws.neon.tech/TravelTide"

# Connection properties
properties = { "user": "Test",
              "password": "bQNxVzJL4g6u",
              "driver": "org.postgresql.Driver"
              }

### 3. Load Data

#### 3.1 Load Metadata

In [ ]:
# Load metadata into Spark DataFrames
traveltide_metadata = spark.read.jdbc(
    url=traveltide_url,
    table="information_schema.tables",
    properties=properties
)

# Filter to only public tables
public_tables = traveltide_metadata.filter(
    traveltide_metadata.table_schema == "public"
)

public_tables.select("table_name").show(truncate=False)


+----------+
|table_name|
+----------+
|sessions  |
|flights   |
|users     |
|hotels    |
+----------+



**Note**: The database contains four main tables relevant for the analysis:

- **sessions** - user sessions, clicks, booking and cancellation flags
- **users** - user demographic and profile information
- **flights** - flight details associated with trips
- **hotels** - hotel details associated with trips

These were identified by querying the `information_schema.tables` filtered on the `public` schema.

In [ ]:
# JDBC read tuning
# Number of partitions for parallel JDBC reads
N_PARTITIONS = 16
# Number of rows fetched per JDBC round-trip
JDBC_FETCHSIZE = "20000"

# bounds per user_id (da users)
# Read min and max user_id from the database
# Used later as lowerBound and upperBound for partitioned JDBC reads
bounds_row = (
    spark.read.format("jdbc")
    .options(
        url=traveltide_url,
        dbtable="(SELECT MIN(user_id) AS min_id, MAX(user_id) AS max_id FROM users) t",
        fetchsize=JDBC_FETCHSIZE,
        **properties
    )
    .load()
    .first()
)

# Convert bounds to Python integers
USER_ID_MIN = int(bounds_row["min_id"])
USER_ID_MAX = int(bounds_row["max_id"])
# Log bounds for validation
print("user_id bounds:", USER_ID_MIN, USER_ID_MAX)


user_id bounds: 0 1020925


**Note**:
Tables are read using JDBC partitioning to improve execution time by enabling parallel reads and reducing overall query latency.

#### 3.2 Load Raw Tables Into Spark DataFrames

In [ ]:
# Define functions to read data from JDBC
# - partitioned by user_id
def read_jdbc_user_partitioned(table_name: str):
    return (
        spark.read.format("jdbc")
        .options(
            url=traveltide_url,
            dbtable=table_name,
            partitionColumn="user_id",
            lowerBound=str(USER_ID_MIN),
            upperBound=str(USER_ID_MAX),
            numPartitions=str(N_PARTITIONS),
            fetchsize=JDBC_FETCHSIZE,
            **properties
        )
        .load()
    )
# - simple (no partitioning)
def read_jdbc_simple(table_name: str):
    return (
        spark.read.format("jdbc")
        .options(
            url=traveltide_url,
            dbtable=table_name,
            numPartitions="2",
            fetchsize=JDBC_FETCHSIZE,
            **properties
        )
        .load()
    )

# users + sessions: partitioned
users_spark    = read_jdbc_user_partitioned("users")
sessions_spark = read_jdbc_user_partitioned("sessions")

# flights + hotels: simple
flights_spark  = read_jdbc_simple("flights")
hotels_spark   = read_jdbc_simple("hotels")


**Note**: The four core tables are loaded via JDBC into Spark DataFrames to enable SQL-based exploration at scale.  
At this stage, data is kept at full granularity to validate schema, keys, and lifecycle logic before applying any cohort filters.  
Cohort selection and session-level extraction are performed only in the final step of the notebook.

In [ ]:
# Check partition only on sessions
sessions_spark.explain(True)

== Parsed Logical Plan ==
Relation [session_id#13193,user_id#13194,trip_id#13195,session_start#13196,session_end#13197,flight_discount#13198,hotel_discount#13199,flight_discount_amount#13200,hotel_discount_amount#13201,flight_booked#13202,hotel_booked#13203,page_clicks#13204,cancellation#13205] JDBCRelation(sessions) [numPartitions=16]

== Analyzed Logical Plan ==
session_id: string, user_id: int, trip_id: string, session_start: timestamp, session_end: timestamp, flight_discount: boolean, hotel_discount: boolean, flight_discount_amount: decimal(38,18), hotel_discount_amount: decimal(38,18), flight_booked: boolean, hotel_booked: boolean, page_clicks: int, cancellation: boolean
Relation [session_id#13193,user_id#13194,trip_id#13195,session_start#13196,session_end#13197,flight_discount#13198,hotel_discount#13199,flight_discount_amount#13200,hotel_discount_amount#13201,flight_booked#13202,hotel_booked#13203,page_clicks#13204,cancellation#13205] JDBCRelation(sessions) [numPartitions=16]

==

#### 3.3 Register Temporary Views for SQL

In [ ]:
# Create temp views for each table
users_spark.createOrReplaceTempView("users")
flights_spark.createOrReplaceTempView("flights")
hotels_spark.createOrReplaceTempView("hotels")
sessions_spark.createOrReplaceTempView("sessions")

#### 3.3 Quick Data Preview

In [ ]:
%sql
-- Count the number of records in Users, Sessions, Flights and Hotels
SELECT (SELECT COUNT(user_id) FROM users) AS users_count,
(SELECT COUNT(session_id) FROM sessions) AS sessions_count,
(SELECT COUNT(trip_id) FROM flights) AS flights_count,
(SELECT COUNT(trip_id) FROM hotels) AS hotels_count;

users_count,sessions_count,flights_count,hotels_count
1020926,5408063,1901038,1918617


**Note**: There are over 5 million records in session table and over one million users. Flights and hotels tables have almost 2 million records

In [ ]:
%sql
-- Display the first 10 rows of sessions
SELECT * FROM sessions LIMIT 10;

session_id,user_id,trip_id,session_start,session_end,flight_discount,hotel_discount,flight_discount_amount,hotel_discount_amount,flight_booked,hotel_booked,page_clicks,cancellation
190-98c09a3b41e6480092f2834685b99872,190,190-5befdd979c734b698ab47f416a4f1f9b,2023-06-10T18:36:00.000Z,2023-06-10T18:58:16.000Z,false,false,null,null,true,true,180,false
369-05dfdf0732034f02be3133950463c061,369,null,2023-06-10T21:38:00.000Z,2023-06-10T21:39:03.000Z,false,false,null,null,false,false,9,false
437-4c352bdebd434bc7b38dc5c0cf2d499a,437,null,2023-06-10T16:24:00.000Z,2023-06-10T16:24:38.000Z,false,false,null,null,false,false,5,false
450-4b6467e93fea4aa6afc7505e643da323,450,null,2023-06-10T17:28:00.000Z,2023-06-10T17:28:46.000Z,true,false,0.300000000000000000,null,false,false,6,false
898-eb22bdaf6c2b402681ed6ef8ae902e70,898,null,2023-06-10T19:38:00.000Z,2023-06-10T19:38:16.000Z,false,false,null,null,false,false,2,false
942-ff5db1d5dcab419e8ff8f7b46b7764f6,942,null,2023-06-10T17:35:00.000Z,2023-06-10T17:36:14.000Z,false,false,null,null,false,false,10,false
1346-11c14dcf3ead4dc6b06b576cff7cbf04,1346,1346-444f3204fcca4a649c8e3abcca044eb4,2023-06-10T08:57:00.000Z,2023-06-10T08:58:58.000Z,false,false,null,null,true,false,16,false
1354-d727b304ece54a24a82a52f9c3c01d5f,1354,1354-bc0aa61994094345b381b6fecb46c7f5,2023-06-10T07:05:00.000Z,2023-06-10T07:06:28.000Z,false,false,null,null,true,false,12,false
1679-f9c76fba94454db18df6d5259ecd0e37,1679,null,2023-06-10T19:02:00.000Z,2023-06-10T19:05:11.000Z,true,true,0.100000000000000000,0.050000000000000000,false,false,26,false
1975-52337d053d7b46b29cf60e9ea7824c30,1975,1975-bae4bfbd12a947cca1406f113f0f37dc,2023-06-10T14:51:00.000Z,2023-06-10T14:53:26.000Z,false,true,null,0.050000000000000000,true,true,20,false


In [ ]:
%sql
-- Display the first 10 rows of users
SELECT *
FROM users
LIMIT 10;

user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date
0,1990-01-22,F,false,false,usa,minneapolis,MSP,44.880000000000000000,-93.217000000000000000,2021-04-01
1,2000-11-08,M,false,false,usa,colorado springs,COS,38.806000000000000000,-104.700000000000000000,2021-04-01
2,1992-09-21,M,false,false,usa,portland,PDX,45.589000000000000000,-122.597000000000000000,2021-04-01
3,1996-11-27,F,false,false,usa,houston,IAH,29.980000000000000000,-95.340000000000000000,2021-04-01
4,1978-01-05,M,true,true,usa,honolulu,HNL,21.316000000000000000,-157.927000000000000000,2021-04-01
5,1989-10-20,F,false,false,canada,edmonton,YED,53.667000000000000000,-113.467000000000000000,2021-04-01
6,1995-10-21,F,false,false,canada,edmonton,YXD,53.573000000000000000,-113.521000000000000000,2021-04-01
7,1983-05-27,M,false,false,usa,fresno,FAT,36.776000000000000000,-119.718000000000000000,2021-04-01
8,2003-05-20,M,false,true,usa,phoenix,PHX,33.434000000000000000,-112.008000000000000000,2021-04-01
9,1977-04-13,M,false,true,usa,las vegas,LAS,36.080000000000000000,-115.152000000000000000,2021-04-01


In [ ]:
%sql
-- Display the first 10 rows of fligths
SELECT *
FROM flights
LIMIT 10;

trip_id,origin_airport,destination,destination_airport,seats,return_flight_booked,departure_time,return_time,checked_bags,trip_airline,destination_airport_lat,destination_airport_lon,base_fare_usd
496435-abe4a3a4d6bc4734a3c5baacbbdd3b2e,AKR,charlotte,CLT,1,true,2023-02-03T15:00:00.000Z,2023-02-05T15:00:00.000Z,0,JetBlue Airways,35.214000000000000000,-80.943000000000000000,118.780000000000000000
496701-690c41b3f3ad44c198bef783bc3bff85,HOU,chicago,ORD,1,true,2023-02-11T12:00:00.000Z,2023-02-17T12:00:00.000Z,1,United Airlines,41.786000000000000000,-87.752000000000000000,268.530000000000000000
496778-89aabe7bd3144c52811a7a61fb05a1de,MSY,florence,FLO,2,true,2023-03-27T20:00:00.000Z,2023-04-08T20:00:00.000Z,1,Allegiant Air,34.185000000000000000,-79.724000000000000000,287.950000000000000000
496839-aca1684059e84783a43698c4bb6ea96b,MDW,los angeles,LAX,1,true,2023-02-08T07:00:00.000Z,2023-02-15T07:00:00.000Z,1,United Airlines,33.942000000000000000,-118.408000000000000000,443.390000000000000000
496871-d3b9674337a049eb91c451a55a81860f,SAT,san jose,SJC,1,true,2023-02-09T08:00:00.000Z,2023-02-12T08:00:00.000Z,1,American Airlines,37.362000000000000000,-121.929000000000000000,417.770000000000000000
496894-c8f1a5bc8b514a1ebd6f760a3e40becb,JFK,austin,AUS,1,false,2023-02-03T16:00:00.000Z,null,1,Southwest Airlines,30.194000000000000000,-97.670000000000000000,229.970000000000000000
496923-ffa36182213749c79022e36f2ab7eac8,HOU,chicago,UGN,1,true,2023-04-24T09:00:00.000Z,2023-05-18T09:00:00.000Z,0,Ryanair,41.786000000000000000,-87.752000000000000000,249.750000000000000000
496924-c51772ff630f409eb3a9ea5a8ceacce0,YZD,seattle,BFI,1,true,2023-02-05T07:00:00.000Z,2023-02-06T07:00:00.000Z,1,Kenmore Air,47.530000000000000000,-122.302000000000000000,594.180000000000000000
496936-8d273e20c3664ba991253d2b2c70edf3,YVR,toronto,YTZ,1,true,2023-02-09T11:00:00.000Z,2023-02-16T11:00:00.000Z,0,Porter Airlines,43.862000000000000000,-79.370000000000000000,614.450000000000000000
497037-eddd4a4ea0cd422e9eb6450e137e98ba,FTW,toronto,YKZ,1,true,2023-02-06T07:00:00.000Z,2023-02-16T07:00:00.000Z,1,American Airlines,43.862000000000000000,-79.370000000000000000,343.030000000000000000


In [ ]:
%sql
-- Display the first 10 rows of hotels
SELECT *
FROM hotels
LIMIT 10;

trip_id,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_per_room_usd
243700-21f0686574264abdba95c975edf0439e,Choice Hotels - fort worth,12,1,2022-07-14T11:00:00.000Z,2022-07-26T11:00:00.000Z,144.000000000000000000
243702-5c9d79726ccd4cd680416769c9001175,NH Hotel - nashville,5,1,2022-07-18T19:32:04.785Z,2022-07-24T11:00:00.000Z,283.000000000000000000
243703-4783bb226e82421e918c9cb927b01950,Starwood - montreal,5,1,2022-07-15T11:00:00.000Z,2022-07-20T11:00:00.000Z,197.000000000000000000
243705-43773b585acd4a1c99ea53a85baa0938,InterContinental - houston,1,1,2022-07-17T12:02:57.300Z,2022-07-19T11:00:00.000Z,578.000000000000000000
243707-42fed4bbc026475aaaf333b1f2993613,NH Hotel - montreal,4,1,2022-07-21T18:43:32.925Z,2022-07-26T11:00:00.000Z,146.000000000000000000
243709-17ef94cbfd394f28aeaa80812463f626,InterContinental - phoenix,0,1,2022-07-16T13:21:21.420Z,2022-07-17T11:00:00.000Z,75.000000000000000000
243710-7be7870893914ff781bb8d698942c90e,Wyndham - washington,6,1,2022-07-17T10:00:09.585Z,2022-07-23T11:00:00.000Z,107.000000000000000000
243712-8fec5eac32074d9390e48fbb3b3fc2ac,Crowne Plaza - san antonio,2,1,2022-07-18T14:59:51.270Z,2022-07-21T11:00:00.000Z,154.000000000000000000
243719-15b49a84d2934c78b034075086b0cb96,Rosewood - san diego,1,1,2022-07-17T15:34:57.090Z,2022-07-19T11:00:00.000Z,99.000000000000000000
243720-aae2fe9145fa4bbf97ea9da37251b74e,Hilton - los angeles,4,2,2022-07-17T18:58:03.945Z,2022-07-22T11:00:00.000Z,225.000000000000000000


**Note**: The first 10 rows of each table confirm consistency in structure. In the `sessions` table, columns like `trip_id`, discount-related fields can be `NULL`, reflecting browsing-only sessions rather than missing data.

### 4. Data Consistency Checks

#### 4.1 Data Types

In [ ]:
%sql
DESCRIBE users;

col_name,data_type,comment
user_id,int,null
birthdate,date,null
gender,string,null
married,boolean,null
has_children,boolean,null
home_country,string,null
home_city,string,null
home_airport,string,null
home_airport_lat,"decimal(38,18)",null
home_airport_lon,"decimal(38,18)",null


In [ ]:
%sql
DESCRIBE sessions;

col_name,data_type,comment
session_id,string,null
user_id,int,null
trip_id,string,null
session_start,timestamp,null
session_end,timestamp,null
flight_discount,boolean,null
hotel_discount,boolean,null
flight_discount_amount,"decimal(38,18)",null
hotel_discount_amount,"decimal(38,18)",null
flight_booked,boolean,null


In [ ]:
%sql
DESCRIBE flights;

col_name,data_type,comment
trip_id,string,null
origin_airport,string,null
destination,string,null
destination_airport,string,null
seats,int,null
return_flight_booked,boolean,null
departure_time,timestamp,null
return_time,timestamp,null
checked_bags,int,null
trip_airline,string,null


In [ ]:
%sql
DESCRIBE hotels;

col_name,data_type,comment
trip_id,string,null
hotel_name,string,null
nights,int,null
rooms,int,null
check_in_time,timestamp,null
check_out_time,timestamp,null
hotel_per_room_usd,"decimal(38,18)",null


**Note**: Column data types were inspected using `DESCRIBE` statements.  
Timestamps, booleans, and numeric fields are consistent with the expected semantics of sessions, users, and products.  
This confirms that the dataset is structurally suitable for reliable joins and lifecycle analysis.

#### 4.2 Primary Keys and Duplicates

In [ ]:
%sql
-- Check duplicated on primary key for sessions
SELECT session_id, COUNT(*) AS duplicates
FROM sessions
GROUP BY session_id
HAVING COUNT(*) > 1;

session_id,duplicates


In [ ]:
%sql
-- Check duplicated on primary key for users
SELECT user_id, COUNT(*) AS duplicates
FROM users
GROUP BY user_id
HAVING COUNT(*) > 1;

user_id,duplicates


In [ ]:
%sql
-- Check duplicated on primary key for flights
SELECT trip_id, COUNT(*) AS duplicates
FROM flights
GROUP BY trip_id
HAVING COUNT(*) > 1;

trip_id,duplicates


In [ ]:
%sql
-- Check duplicated on primary key for hotels
SELECT trip_id, COUNT(*) AS duplicates
FROM hotels
GROUP BY trip_id
HAVING COUNT(*) > 1;

trip_id,duplicates


**Note**: Primary keys were validated for uniqueness across all tables:  
`users.user_id`, `sessions.session_id`, `flights.trip_id`, and `hotels.trip_id`.  

No duplicates were detected, ensuring that joins do not introduce artificial row duplication.

#### 4.3 Null Values and Basic Statistics

###### Sessions

In [ ]:
%sql
-- Check null values on sessions columns
SELECT
    COUNT(CASE WHEN session_id IS NULL THEN 1 END) AS session_id_nulls,
    COUNT(CASE WHEN trip_id IS NULL THEN 1 END) AS trip_id_nulls,
    COUNT(CASE WHEN session_start IS NULL THEN 1 END) AS session_start_nulls,
    COUNT(CASE WHEN session_end IS NULL THEN 1 END) AS session_end_nulls,
    COUNT(CASE WHEN page_clicks IS NULL THEN 1 END) AS page_clicks_nulls,
    COUNT(CASE WHEN flight_discount IS NULL THEN 1 END) AS flight_discount_nulls,
    COUNT(CASE WHEN flight_discount_amount IS NULL THEN 1 END) AS flight_discount_amount_nulls,
    COUNT(CASE WHEN flight_booked IS NULL THEN 1 END) AS flight_booked_nulls,
    COUNT(CASE WHEN hotel_discount IS NULL THEN 1 END) AS hotel_discount_nulls,
    COUNT(CASE WHEN hotel_discount_amount IS NULL THEN 1 END) AS hotel_discount_amount_nulls,
    COUNT(CASE WHEN hotel_booked IS NULL THEN 1 END) AS hotel_booked_nulls,
    COUNT(CASE WHEN cancellation IS NULL THEN 1 END) AS cancellation_nulls
FROM sessions;


session_id_nulls,trip_id_nulls,session_start_nulls,session_end_nulls,page_clicks_nulls,flight_discount_nulls,flight_discount_amount_nulls,flight_booked_nulls,hotel_discount_nulls,hotel_discount_amount_nulls,hotel_booked_nulls,cancellation_nulls
0,3072218,0,0,0,0,4522267,0,0,4716683,0,0


**Note**: Missing values are concentrated in `trip_id` and booking-related fields.  
These NULLs are structurally expected and correspond to browsing-only sessions without an associated trip.  
No unexpected data loss is observed in session records.

###### Users

In [ ]:
%sql
-- Check null values on users columns
SELECT
    COUNT(CASE WHEN user_id IS NULL THEN 1 END) AS user_id_nulls,
    COUNT(CASE WHEN birthdate IS NULL THEN 1 END) AS birthdate_nulls,
    COUNT(CASE WHEN gender IS NULL THEN 1 END) AS gender_nulls,
    COUNT(CASE WHEN married IS NULL THEN 1 END) AS married_nulls,
    COUNT(CASE WHEN has_children IS NULL THEN 1 END) AS has_children_nulls,
    COUNT(CASE WHEN home_country IS NULL THEN 1 END) AS home_country_nulls,
    COUNT(CASE WHEN home_city IS NULL THEN 1 END) AS home_city_nulls,
    COUNT(CASE WHEN home_airport IS NULL THEN 1 END) AS home_airport_nulls,
    COUNT(CASE WHEN home_airport_lat IS NULL THEN 1 END) AS home_airport_lat_nulls,
    COUNT(CASE WHEN home_airport_lon IS NULL THEN 1 END) AS home_airport_lon_nulls,
    COUNT(CASE WHEN sign_up_date IS NULL THEN 1 END) AS sign_up_date_nulls
FROM users;


user_id_nulls,birthdate_nulls,gender_nulls,married_nulls,has_children_nulls,home_country_nulls,home_city_nulls,home_airport_nulls,home_airport_lat_nulls,home_airport_lon_nulls,sign_up_date_nulls
0,0,0,0,0,0,0,0,0,0,0


**Note**: The users table has no NULL values in the columns.

###### Flights

In [ ]:
%sql
-- Check null values on flights columns
SELECT
    COUNT(CASE WHEN trip_id IS NULL THEN 1 END) AS trip_id_nulls,
    COUNT(CASE WHEN origin_airport IS NULL THEN 1 END) AS origin_airport_nulls,
    COUNT(CASE WHEN destination IS NULL THEN 1 END) AS destination_nulls,
    COUNT(CASE WHEN destination_airport IS NULL THEN 1 END) AS destination_airport_nulls,
    COUNT(CASE WHEN seats IS NULL THEN 1 END) AS seats_nulls,
    COUNT(CASE WHEN return_flight_booked IS NULL THEN 1 END) AS return_flight_booked_nulls,
    COUNT(CASE WHEN departure_time IS NULL THEN 1 END) AS departure_time_nulls,
    COUNT(CASE WHEN return_time IS NULL THEN 1 END) AS return_time_nulls,
    COUNT(CASE WHEN checked_bags IS NULL THEN 1 END) AS checked_bags_nulls,
    COUNT(CASE WHEN trip_airline IS NULL THEN 1 END) AS trip_airline_nulls,
    COUNT(CASE WHEN destination_airport_lat IS NULL THEN 1 END) AS destination_airport_lat_nulls,
    COUNT(CASE WHEN destination_airport_lon IS NULL THEN 1 END) AS destination_airport_lon_nulls,
    COUNT(CASE WHEN base_fare_usd IS NULL THEN 1 END) AS base_fare_usd_nulls
FROM flights;


trip_id_nulls,origin_airport_nulls,destination_nulls,destination_airport_nulls,seats_nulls,return_flight_booked_nulls,departure_time_nulls,return_time_nulls,checked_bags_nulls,trip_airline_nulls,destination_airport_lat_nulls,destination_airport_lon_nulls,base_fare_usd_nulls
0,0,0,0,0,0,0,88734,0,0,0,0,0


**Note**: `return_time` is NULL for one-way flights. All other flight fields are populated, indicating complete booking information for flights.

###### Hotels

In [ ]:
%sql
-- Check null values on nights columns
SELECT
    COUNT(CASE WHEN trip_id IS NULL THEN 1 END) AS trip_id_nulls,
    COUNT(CASE WHEN hotel_name IS NULL THEN 1 END) AS hotel_name_nulls,
    COUNT(CASE WHEN nights IS NULL THEN 1 END) AS nights_nulls,
    COUNT(CASE WHEN rooms IS NULL THEN 1 END) AS rooms_nulls,
    COUNT(CASE WHEN check_in_time IS NULL THEN 1 END) AS check_in_time_nulls,
    COUNT(CASE WHEN check_out_time IS NULL THEN 1 END) AS check_out_time_nulls,
    COUNT(CASE WHEN hotel_per_room_usd IS NULL THEN 1 END) AS hotel_per_room_usd_nulls
FROM hotels;

trip_id_nulls,hotel_name_nulls,nights_nulls,rooms_nulls,check_in_time_nulls,check_out_time_nulls,hotel_per_room_usd_nulls
0,0,0,0,0,0,0


**Note**: The hotels table has no NULL values in the columns.

### 5. Exploratory Analysis

This section provides a light structured exploratory analysis of the TravelTide tables, with a focus on understanding user behavior, session patterns, and early signals of trip lifecycle dynamics.
The goal is to surface insights that justify later modeling and cohort choices.

#### 5.1 Sessions Table Exploration

##### Total Sessions per User

In [ ]:
%sql
-- Total sessions per user
SELECT user_id, COUNT(*) AS tot_sessions
FROM sessions
GROUP BY user_id
ORDER BY tot_sessions DESC
LIMIT 20;


user_id,tot_sessions
36751,17
5655,16
24936,16
6914,16
58289,16
63888,16
29473,16
3389,16
22587,16
49389,16


**Note**: The upper tail of the session count distribution was inspected to identify potential extreme user behavior.  
No evident anomalies appear among the most active users, supporting the use of session volume as a valid engagement signal.

##### Cancellation Distribution

In [ ]:
%sql
-- Cancellation distribution in sessions
SELECT cancellation, COUNT(*) AS num_sessions
FROM sessions
GROUP BY cancellation;


cancellation,num_sessions
true,90670
false,5317393


**Note**: Cancellation sessions represent a small fraction of total sessions.  
This indicates that cancellations are sparse events and should be handled explicitly when reconstructing trip lifecycles.

##### Distinguishing Booking, Browsing, and Cancellation Sessions

In [ ]:
%sql
-- Booking, browsing, and cancellation Sessions
-- Classify sessions based on booking intent and outcome.
SELECT
sum(CASE WHEN (cancellation = false) and (flight_booked=true or hotel_booked=true) THEN 1 ELSE 0 END) as booking_session,
sum(CASE WHEN (cancellation = false) and flight_booked=false and hotel_booked=false THEN 1 ELSE 0 END) as browsing_sessions,
sum(CASE WHEN (cancellation = true) THEN 1 ELSE 0 END) as cancellation_sessions
FROM sessions


booking_session,browsing_sessions,cancellation_sessions
2245175,3072218,90670


**Note**: The number of browsing_sessions (3,072,218) exactly matches the number of sessions where `trip_id IS NULL`.
This confirms that pure browsing sessions do not have an associated trip, while booking and cancellation sessions do (`trip_id IS NOT NULL`).

This distinction allows us to separate sessions with trips from browsing-only sessions.

##### Trip vs Browsing-Only Sessions

In [ ]:
%sql
-- Trip vs Browsing-Only Sessions
-- Validate the relationship between session intent and presence of trip_id.
SELECT CASE WHEN trip_id IS NULL THEN 'no_trip' ELSE 'with_trip' END AS trip_flag,
       COUNT(*) AS num_sessions
FROM sessions
GROUP BY trip_flag;

trip_flag,num_sessions
no_trip,3072218
with_trip,2335845


**Note**: This confirms the previous observation: all sessions with a trip have `trip_id` populated, while browsing-only sessions do not.

At this stage, three types of sessions are clearly identified: **browsing-only**, **booking**, and **cancellation**.

These distinctions set the stage for examining whether cancellations share the same `trip_id` as previous booking sessions, for example, to reconstruct the lifecycle of a trip.

##### Booking–Cancellation Relationship

In [ ]:
%sql
-- Test whether cancellation sessions are linked to prior booking sessions via trip_id.
-- Identify trips that appear in cancellation sessions
WITH cancelled_trips AS (
    SELECT DISTINCT trip_id
    FROM sessions
    WHERE cancellation = true
      AND trip_id IS NOT NULL
)

-- Count how many booking sessions share the same trip_id
SELECT
    COUNT(DISTINCT s.trip_id) AS booking_and_cancelled_trips,
    SUM(CASE WHEN s.cancellation = false THEN 1 ELSE 0 END) AS booking_sessions_count,
    SUM(CASE WHEN s.cancellation = true THEN 1 ELSE 0 END) AS cancellation_sessions_count
FROM sessions s
JOIN cancelled_trips ct
  ON s.trip_id = ct.trip_id;

booking_and_cancelled_trips,booking_sessions_count,cancellation_sessions_count
90670,90670,90670


**Note**: Cancelled trips exhibit two sessions linked by the same trip_id:
**one booking session and one cancellation session**.

This confirms a stable and interpretable trip lifecycle, which is later used to classify sessions
and compute booking and cancellation metrics correctly.

Trips follow a simple and consistent lifecycle:
- **browsing sessions** have no `trip_id`,
- **booking sessions** create a trip (`trip_id`, `cancellation = FALSE`),
- **cancellation sessions** refer back to the same `trip_id` (`cancellation = TRUE`).

This structure clearly distinguishes browsing, booking, and cancellation behavior
and enables coherent session-level extraction without inflating booking counts.

##### Flight/Hotel booking flags in Booking Sessions

In [ ]:
%sql
-- Analyze flight and hotel booking flags in active sessions
SELECT
  flight_booked,
  hotel_booked,
  COUNT(*) AS num_sessions
FROM sessions
WHERE cancellation = false
GROUP BY flight_booked, hotel_booked
ORDER BY flight_booked DESC, hotel_booked DESC;


flight_booked,hotel_booked,num_sessions
true,true,1574480
true,false,326558
false,true,344137
false,false,3072218


**Note**: This query counts only non-cancellation sessions (cancellation = false) to capture the real booking activity.

- `flight_booked = true` AND `hotel_booked = true` are sessions where both flight and hotel were booked (1,574,480).

- `flight_booked = true` AND `hotel_booked = false` are flight-only bookings (326,558).

- `flight_booked = false` AND `hotel_booked = true` are hotel-only bookings (344,137).

- `flight_booked = false` AND `hotel_booked = false` are browsing-only sessions with no booking (3,072,218).

**Observation**:
The count of browsing-only sessions (`false, false`) exactly matches the number of sessions with `trip_id = NULL` identified previously, confirming that sessions without a trip correspond to pure browsing activity.

In [ ]:
%sql
-- Check for cancellation sessions with incomplete bookings
SELECT *
FROM sessions
WHERE cancellation = true
  AND (hotel_booked = false OR flight_booked = false);

session_id,user_id,trip_id,session_start,session_end,flight_discount,hotel_discount,flight_discount_amount,hotel_discount_amount,flight_booked,hotel_booked,page_clicks,cancellation


**Note**: This query checks whether there are any cancellation sessions where either `flight_booked` or `hotel_booked` is `false`.

The result, no rows are returned, confirms that all cancellation sessions always have both `flight_booked` and `hotel_booked` set to `true`, regardless of whether the original booking included only a flight, only a hotel, or both.

From a session-level perspective, cancellation flags are standardized; the individual flags no longer reflect the original booking composition. Therefore, when analyzing booking behavior, we must filter out `cancellation = true` to understand the actual booking patterns.

##### Session Details for Trip Shared across Multiple Sessions

In [ ]:
%sql
-- Get all session details for trips shared across multiple sessions
--Identify trips with more than one session
WITH trips_multiple_sessions AS (
    SELECT trip_id
    FROM sessions
    WHERE trip_id IS NOT NULL
    GROUP BY trip_id
    HAVING COUNT(*) > 1
)
-- Retrieve all session records for these trips
SELECT s.*
FROM sessions s
JOIN trips_multiple_sessions tms
    ON s.trip_id = tms.trip_id
ORDER BY s.trip_id, s.session_start
LIMIT 10;

session_id,user_id,trip_id,session_start,session_end,flight_discount,hotel_discount,flight_discount_amount,hotel_discount_amount,flight_booked,hotel_booked,page_clicks,cancellation
1-5da1c1879f5440038967d1405e414ffd,1,1-f35a49334eaa473aa993bde2c8f670ec,2022-02-26T14:19:00.000Z,2022-02-26T14:21:10.000Z,false,false,null,null,true,true,17,false
1-18ef3048214646d8a0d46748b4136401,1,1-f35a49334eaa473aa993bde2c8f670ec,2022-05-31T14:21:10.000Z,2022-05-31T14:46:44.356Z,true,true,null,null,true,true,25,true
100-9a9cb6d0c6584431bd1b59e517e1d60e,100,100-4d42c16dea8441d391a44b1bdddf238b,2021-04-05T16:03:00.000Z,2021-04-05T16:05:28.000Z,false,false,null,null,true,true,20,false
100-e35f4049603f450fa4c83391edcf6580,100,100-4d42c16dea8441d391a44b1bdddf238b,2021-07-16T16:05:28.000Z,2021-07-16T18:05:28.000Z,true,true,null,null,true,true,128,true
1000-de3ba74e7d234d908e36873f69348bd5,1000,1000-824d298be9454790bfa01be81c6bef8d,2021-06-17T15:12:00.000Z,2021-06-17T15:25:50.000Z,false,false,null,null,true,true,111,false
1000-843ed82dc9464c35891ed43819e9eed6,1000,1000-824d298be9454790bfa01be81c6bef8d,2021-06-17T15:25:50.000Z,2021-06-17T17:25:50.000Z,true,true,null,null,true,true,200,true
10001-88a731e9408c49cea4339158a3e59a42,10001,10001-2a94b4fa9e834e9f8703ba8ac65b5bec,2021-06-22T22:53:00.000Z,2021-06-22T22:54:27.000Z,true,false,0.150000000000000000,null,true,false,12,false
10001-477c288b465046c49fca4db38c89d517,10001,10001-2a94b4fa9e834e9f8703ba8ac65b5bec,2021-10-31T22:54:27.000Z,2021-11-01T00:54:27.000Z,true,true,null,null,true,true,200,true
100012-211fb2ac2a6941fc8e8e61720d50b2a1,100012,100012-b58d1cc44588426189f46053b68cd4ca,2022-03-07T08:00:00.000Z,2022-03-07T08:02:00.000Z,false,false,null,null,true,false,16,false
100012-6fbecf5f9b484fdd9a715b0a3259d2b5,100012,100012-b58d1cc44588426189f46053b68cd4ca,2022-06-08T08:02:00.000Z,2022-06-08T10:02:00.000Z,true,true,null,null,true,true,197,true


**Note**: Each cancelled trip has exactly two sessions: a booking session and a cancellation session. Cancellation sessions always have all booking flags set to TRUE; to see the actual booking details, refer to the original booking session for the same trip_id.

The behavior of cancellation sessions, in terms of the boolean flags (`flight_booked`, `hotel_booked`, `flight_discount`, `hotel_discount`), will be examined in detail in the session-level notebook.

This deeper analysis will clarify which flags are actually meaningful during cancellations and how to filter the data to capture user behavior in these sessions accurately.

##### Session Duration Distribution

In [ ]:
%sql
-- Distribution of session length
-- Understand how long users typically spend per session.
SELECT
    CASE
        WHEN session_length_minutes BETWEEN 0 AND 5 THEN '0-5 min'
        WHEN session_length_minutes BETWEEN 6 AND 10 THEN '6-10 min'
        WHEN session_length_minutes BETWEEN 11 AND 30 THEN '11-30 min'
        WHEN session_length_minutes BETWEEN 31 AND 60 THEN '31-60 min'
        ELSE '>60 min'
    END AS session_length_bucket,
    COUNT(*) AS num_sessions
FROM (
    SELECT
        CAST((CAST(session_end AS TIMESTAMP) - CAST(session_start AS TIMESTAMP)) AS BIGINT) / 60 AS session_length_minutes
    FROM sessions
) AS sub
GROUP BY session_length_bucket
ORDER BY
    CASE
        WHEN session_length_bucket = '0-5 min' THEN 1
        WHEN session_length_bucket = '6-10 min' THEN 2
        WHEN session_length_bucket = '11-30 min' THEN 3
        WHEN session_length_bucket = '31-60 min' THEN 4
        ELSE 5
    END;


session_length_bucket,num_sessions
0-5 min,5053673
6-10 min,128281
11-30 min,28934
31-60 min,11487
>60 min,185688


**Note**: Most sessions (5.05M) are very short, lasting between 0-5 minutes, reflecting typical browsing-only activity. Longer sessions are progressively less frequent: 6-10 minutes (128k), 11-30 minutes (29k), and 31-60 minutes (11k).

Interestingly, a noticeable portion of sessions (>60 minutes, 186k) likely corresponds to more involved activity such as booking trips or researching multiple options.

This distribution highlights the diversity in user engagement and supports the distinction between quick browsing sessions and full booking sessions in further analysis.

In [ ]:
%sql
-- Count page clicks in session
SELECT page_clicks, COUNT(*) AS count
FROM sessions
GROUP BY page_clicks
ORDER BY page_clicks;

page_clicks,count
0,7
1,40921
2,183693
3,260132
4,274756
5,260093
6,234807
7,207309
8,186882
9,186806


**Note**: The distribution highlights the presence of a large number of browsing-only sessions,
which explains the high frequency of NULL values in `trip_id` and booking-related fields observed earlier.

##### Discount Offered Flags

In [ ]:
%sql
SELECT
    flight_discount,
    hotel_discount,
    COUNT(*) AS num_sessions
FROM sessions
WHERE cancellation = false
GROUP BY flight_discount, hotel_discount
ORDER BY flight_discount DESC, hotel_discount DESC;

flight_discount,hotel_discount,num_sessions
true,true,137032
true,false,748764
false,true,554348
false,false,3877249


**Note**: This table shows the distribution of sessions according to whether flight and hotel discounts were offered. Only non-cancellation sessions are considered to avoid distortion caused by cancellation sessions, where all discount flags are automatically set to TRUE.

- Most sessions (3.87M) did not have any discounts applied.

- Flight-only discounts appear in ~748k sessions, hotel-only discounts in 554k sessions.

- Both flight and hotel discounts were offered in 137k sessions.

This gives us a realistic picture of promotional exposure across active user sessions, which will be relevant for understanding user behavior and price sensitivity in later analyses.

In [ ]:
%sql
SELECT
    u.gender,

    COUNT(*) AS total_sessions,

    SUM(
        CASE
            WHEN s.cancellation = FALSE
             AND (s.flight_booked = TRUE OR s.hotel_booked = TRUE)
            THEN 1 ELSE 0
        END
    ) AS booking_sessions,

    SUM(
        CASE
            WHEN s.cancellation = TRUE
            THEN 1 ELSE 0
        END
    ) AS cancellation_sessions,

    -- Conversion rate: bookings / total sessions
    ROUND(
        SUM(
            CASE
                WHEN s.cancellation = FALSE
                 AND (s.flight_booked = TRUE OR s.hotel_booked = TRUE)
                THEN 1 ELSE 0
            END
        ) * 1.0 / COUNT(*),
        3
    ) AS booking_rate,

    -- Cancellation rate: cancellations / booking sessions
    ROUND(
        SUM(
            CASE
                WHEN s.cancellation = TRUE
                THEN 1 ELSE 0
            END
        ) * 1.0 /
        NULLIF(
            SUM(
                CASE
                    WHEN s.cancellation = FALSE
                     AND (s.flight_booked = TRUE OR s.hotel_booked = TRUE)
                    THEN 1 ELSE 0
                END
            ),
            0
        ),
        3
    ) AS cancellation_rate

FROM sessions s
JOIN users u
  ON s.user_id = u.user_id

GROUP BY u.gender
ORDER BY total_sessions DESC;


gender,total_sessions,booking_sessions,cancellation_sessions,booking_rate,cancellation_rate
M,2722001,1162748,44789,0.427,0.039
F,2645784,1065060,45227,0.403,0.042
O,40278,17367,654,0.431,0.038


In [ ]:
%sql
SELECT
    gender,
    MIN(session_start) AS first_session_date,
    MAX(session_start) AS last_session_date,
    COUNT(DISTINCT s.user_id) AS n_users
FROM sessions s
JOIN users u ON s.user_id = u.user_id
GROUP BY gender
ORDER BY gender;


gender,first_session_date,last_session_date,n_users
F,2021-04-01T04:57:00.000Z,2023-07-28T23:24:52.000Z,453654
M,2021-04-01T00:42:00.000Z,2023-07-28T23:57:55.000Z,558986
O,2021-04-11T14:54:00.000Z,2023-07-28T18:17:47.000Z,8286


#### 5.2 Users Table Exploration

##### Distribution by Gender, Marital Status, and Children

In [ ]:
%sql
-- Distribution by gender, marital status, and children
SELECT gender, married, has_children, COUNT(user_id) AS tot_users
FROM users
GROUP BY gender, married, has_children;

gender,married,has_children,tot_users
M,true,true,82754
M,false,true,91457
F,true,false,110504
M,true,false,127701
O,false,true,1659
O,true,true,670
F,false,true,70928
M,false,false,257074
F,true,true,72169
O,true,false,1019


**Note**: The user base is diverse across gender, marital status, and parental status.

- Male users (M) represent a slightly larger segment than female users (F), while O (Other/unspecified) is very small.

- Most users without children are single or married (M false/true), while users with children are more balanced across gender and marital status.

- This distribution highlights that family-related features (has_children and married) could be meaningful in customer segmentation.

- No extreme outliers are apparent in user counts per category, suggesting a reasonably balanced demographic representation.

##### Age at Travel Distribution

In [ ]:
%sql
-- Age distribution retrieved from users based on date of Travel
SELECT
  YEAR(u.birthdate) AS birthyear,
  FLOOR(datediff(f.departure_time, u.birthdate) / 365) AS age_at_travel,
  COUNT(DISTINCT u.user_id) AS tot_users
FROM users u
JOIN sessions s ON s.user_id = u.user_id
JOIN flights f ON f.trip_id = s.trip_id
GROUP BY
  YEAR(u.birthdate),
  FLOOR(datediff(f.departure_time, u.birthdate) / 365)
ORDER BY birthyear DESC;

birthyear,age_at_travel,tot_users
2006,14,194
2006,15,3359
2006,16,14421
2006,18,636
2006,17,10249
2005,19,114
2005,15,38
2005,16,608
2005,17,2431
2005,18,1719


**Note**: The age-at-travel distribution, useful for cohort analysis,  shows a strong concentration in the adult population, with the highest number of users clustered between approximately 25 and 55 years old. Younger users (below 18) and older age groups (above 70) are present but represent a significantly smaller share of the total population.

Overall, the distribution appears smooth and consistent across birth cohorts, suggesting no major anomalies in the age calculation and a user base primarily composed of working-age travelers.

In [ ]:
%sql
SELECT home_country, COUNT(user_id) as tot_users
FROM users
GROUP BY home_country;

home_country,tot_users
canada,172572
usa,848354


**Note**: The user distribution is strongly skewed toward North America, with most users coming from the United States and a smaller but still significant share from Canada. This geographic  concentration may provide useful context for interpreting user behavior in subsequent analyses.

#### 5.3 Flights and Hotels Exploration

##### Most used airlines in the last 6 months

In [ ]:
%sql
-- Most used airlines in the last 6 months
SELECT trip_airline, COUNT(trip_id) AS tot_used
FROM flights
WHERE departure_time > (SELECT MAX(departure_time) - INTERVAL '6 months' FROM flights)
GROUP BY trip_airline
ORDER BY tot_used DESC
LIMIT 30;

trip_airline,tot_used
American Airlines,2590
Delta Air Lines,2437
United Airlines,2288
Ryanair,1731
Southwest Airlines,987
JetBlue Airways,785
Allegiant Air,731
Air Canada,727
AirTran Airways,592
Air India Limited,549


**Note**: The airline usage distribution is highly skewed toward a small set of major carriers, with American Airlines, Delta Air Lines, and United Airlines being the most frequently used.

Low-cost and regional airlines are also well represented, indicating a diverse mix of travel patterns and user preferences across both full-service and budget carriers.

##### Average Seats by Airline

In [ ]:
%sql
-- average seats used by airline
SELECT trip_airline, AVG(seats) AS average_seats
FROM flights
GROUP BY trip_airline
ORDER BY average_seats DESC;

trip_airline,average_seats
Binter Canarias,3.3333333333333335
LSM Airlines,3.1666666666666665
Helvetic Airways,3.0
Air Greenland,3.0
Hapagfly,2.6666666666666665
Israir,2.5
Kam Air,2.5
Air Bosna,2.5
Arkia Israel Airlines,2.4615384615384617
TRIP Linhas A,2.4545454545454546


**Note**: Airlines with a lower average number of seats per trip are mostly associated with business travel or bookings for one or two passengers.

In contrast, airlines showing higher average seat counts tend to be preferred for family or group travel, where price sensitivity is higher and more economical carriers are often selected.

##### Hotel Name Distribution

In [ ]:
%sql
-- Hotel name distribution
SELECT hotel_name, COUNT(*) AS tot_hotel_names
FROM hotels
GROUP BY hotel_name
ORDER BY tot_hotel_names DESC;

hotel_name,tot_hotel_names
Extended Stay - new york,14075
Radisson - new york,14073
Starwood - new york,14029
Conrad - new york,14022
Rosewood - new york,14017
Banyan Tree - new york,13974
Best Western - new york,13959
Shangri-La - new york,13958
InterContinental - new york,13956
Aman Resorts - new york,13940


**Note**: Hotel usage is heavily concentrated in major North American cities, with New York, Los Angeles, Toronto, and Chicago showing the highest number of stays across almost all hotel chains. Large international brands (e.g. Marriott, Hilton, Hyatt, Accor) appear consistently across locations, indicating strong brand standardization and user preference for well-known chains. Smaller volumes are observed in international cities, suggesting more localized or less frequent travel compared to domestic destinations.

In [ ]:
%sql
--Annual Flight Counts Based on Departure Year
SELECT
    EXTRACT(YEAR FROM f.departure_time) AS year,
    COUNT(*) AS num_flights
FROM flights f
WHERE f.departure_time IS NOT NULL
GROUP BY 1
ORDER BY 1;

year,num_flights
2021,83346
2022,634052
2023,1145121
2024,38519


**Note**: The distribution of flight departures is strongly concentrated in 2023, which contains the majority of observed records. This reinforces the relevance of focusing the analysis on a recent and well-defined temporal window, consistent with Elena’s reference timeframe. Earlier and later years are present but significantly less represented.

##### Nights Distribution

In [ ]:
%sql
-- Count hotel sessions grouped by hotel nights
SELECT
    nights,
    COUNT(*) AS num_sessions
FROM hotels
GROUP BY nights
ORDER BY nights;

nights,num_sessions
-2,281
-1,11786
0,139952
1,378046
2,342120
3,250151
4,177756
5,128172
6,95335
7,73353


**Note**: The distribution of nights is heavily right-skewed, with the majority of sessions concentrated between 1 and 3 nights. The frequency decreases steadily as the number of nights increases, indicating that short stays are far more common than long ones.

Negative and zero values are present and likely represent data quality issues or edge cases; these will be addressed in the next notebook during data cleaning.

In [ ]:
%sql
-- Annual Hotel Counts Based on Check-in Year
SELECT
    EXTRACT(YEAR FROM h.check_in_time) AS year,
    COUNT(*) AS num_stays
FROM hotels h
WHERE h.check_in_time IS NOT NULL
GROUP BY 1
ORDER BY 1;

year,num_stays
2021,88781
2022,661387
2023,1153086
2024,15363


**Note**: The distribution of hotel check-ins mirrors that of flights, with the majority of records concentrated in 2023. This confirms that the main activity falls within a recent and well-defined temporal window, supporting the focus on this period for further analysis.

#### 5.4 Cohort Sensitivity Analysis (User-Level)

This section evaluates the robustness of the user cohort definition used for subsequent analysis.
In particular, it examines how different minimum-session thresholds affect:

- User representation by gender

- The stability of booking and cancellation behavior

The goal is to ensure that the selected cohort provides sufficient behavioral depth **without introducing systematic bias**.

##### 5.4.1 Baseline Cohort

###### Cohort definition: sessions > 7

In [ ]:
%sql
-- select sessions with session_start > '2023-01-04'
WITH filtered_sessions AS (
  SELECT
    user_id,
    trip_id,
    cancellation,
    flight_booked,
    hotel_booked
  FROM sessions
  WHERE session_start > '2023-01-04'
),

-- select eligible users from filtered_sessions
eligible_users AS (
  SELECT user_id
  FROM filtered_sessions
  GROUP BY user_id
  HAVING COUNT(*) > 7  -- users with more than 7 sessions after 2023-01-04
),

-- select cohort users in filtered_sessions
cohort_sessions AS (
  SELECT fs.*
  FROM filtered_sessions fs
  JOIN eligible_users eu ON eu.user_id = fs.user_id
),

-- select all sessions from cohort sessions
sessions_by_user AS (
  SELECT
    user_id,
    COUNT(*) AS total_sessions,
    COUNT(*) FILTER (WHERE trip_id IS NULL) AS browsing_sessions, -- only browsing sessions
    COUNT(*) FILTER (
      WHERE trip_id IS NOT NULL AND cancellation = FALSE
      AND (flight_booked = TRUE OR hotel_booked = TRUE) -- only booking sessions
    ) AS booking_sessions,
    COUNT(*) FILTER (WHERE cancellation = TRUE) AS cancellation_sessions -- only cancellation sessions
  FROM cohort_sessions
  GROUP BY user_id
),

/* select user by gender from filtered sessions_by_user
now to reduce the query execution time */
users_with_gender AS (
  SELECT
    sbu.*,
    u.gender
  FROM sessions_by_user sbu
  JOIN users u ON u.user_id = sbu.user_id
)

-- select all the information from users_with_gender and perform aggregations
SELECT
  gender,
  COUNT(*) AS n_users,
  SUM(browsing_sessions) AS n_browsings, -- total number of browsing sessions
  SUM(booking_sessions) AS n_bookings,   -- total number of booking sessions
  SUM(cancellation_sessions) AS n_cancels, -- total number of cancellations
  SUM(total_sessions) AS n_sessions, -- total number of sessions
  ROUND(SUM(booking_sessions)::numeric / NULLIF(SUM(total_sessions), 0), 3) AS booking_rate, -- booking rate
  ROUND(SUM(cancellation_sessions)::numeric / NULLIF(SUM(booking_sessions), 0), 3) AS cancel_rate -- cancellation rate
FROM users_with_gender
GROUP BY gender -- group by gender
ORDER BY n_users DESC;


gender,n_users,n_browsings,n_bookings,n_cancels,n_sessions,booking_rate,cancel_rate
F,5292,28741,14224,521,43486,0.327,0.037
M,695,3708,1839,88,5635,0.326,0.048
O,11,60,29,1,90,0.322,0.034


**Note**: This analysis evaluates the sensitivity of user-level metrics to different minimum-session cohort thresholds.

`n_sessions` includes all session types (browsing, booking, cancellation).  
`n_bookings` counts booking sessions only (`trip_id IS NOT NULL` and `cancellation = FALSE`).  
`n_cancels` counts cancellation sessions (`cancellation = TRUE`).


`booking_rate` is computed as `n_bookings / n_sessions`, while `cancellation_rate` is computed as `n_cancels / n_bookings`.

Comparing multiple minimum-activity thresholds shows that absolute gender representation varies with the cohort definition, while booking and cancellation rates remain stable.  
This indicates that the selected cohort (>7 sessions) preserves behavioral validity while prioritizing users with sufficient interaction history.

##### 5.4.2 Lower Threshold Cohort

###### Cohort definition: sessions > 5

In [ ]:
%sql
-- select sessions with session_start > '2023-01-04'
WITH filtered_sessions AS (
  SELECT
    user_id,
    trip_id,
    cancellation,
    flight_booked,
    hotel_booked
  FROM sessions
  WHERE session_start > '2023-01-04'
),

-- select eligible users from filtered_sessions
eligible_users AS (
  SELECT user_id
  FROM filtered_sessions
  GROUP BY user_id
  HAVING COUNT(*) > 5  -- users with more than 5 sessions after 2023-01-04
),

-- select cohort users in filtered_sessions
cohort_sessions AS (
  SELECT fs.*
  FROM filtered_sessions fs
  JOIN eligible_users eu ON eu.user_id = fs.user_id
),

-- select all sessions from cohort sessions
sessions_by_user AS (
  SELECT
    user_id,
    COUNT(*) AS total_sessions,
    COUNT(*) FILTER (WHERE trip_id IS NULL) AS browsing_sessions, -- only browsing sessions
    COUNT(*) FILTER (
      WHERE trip_id IS NOT NULL AND cancellation = FALSE
      AND (flight_booked = TRUE OR hotel_booked = TRUE) -- only booking sessions
    ) AS booking_sessions,
    COUNT(*) FILTER (WHERE cancellation = TRUE) AS cancellation_sessions -- only cancellation sessions
  FROM cohort_sessions
  GROUP BY user_id
),

/* select user by gender from filtered sessions_by_user
now to reduce the query execution time */
users_with_gender AS (
  SELECT
    sbu.*,
    u.gender
  FROM sessions_by_user sbu
  JOIN users u ON u.user_id = sbu.user_id
)

-- select all the information from users_with_gender and perform aggregations
SELECT
  gender,
  COUNT(*) AS n_users,
  SUM(browsing_sessions) AS n_browsings, -- total number of browsing sessions
  SUM(booking_sessions) AS n_bookings,   -- total number of booking sessions
  SUM(cancellation_sessions) AS n_cancels, -- total number of cancellations
  SUM(total_sessions) AS n_sessions, -- total number of sessions
  ROUND(SUM(booking_sessions)::numeric / NULLIF(SUM(total_sessions), 0), 3) AS booking_rate, -- booking rate
  ROUND(SUM(cancellation_sessions)::numeric / NULLIF(SUM(booking_sessions), 0), 3) AS cancel_rate -- cancellation rate
FROM users_with_gender
GROUP BY gender -- group by gender
ORDER BY n_users DESC;


gender,n_users,n_browsings,n_bookings,n_cancels,n_sessions,booking_rate,cancel_rate
F,54456,223412,125967,3499,352878,0.357,0.028
M,23224,89014,54251,1788,145053,0.374,0.033
O,365,1383,877,28,2288,0.383,0.032


**Note**: Lowering the threshold to more than 5 sessions increases user coverage and improves demographic balance.  
However, the additional users contribute primarily browsing activity, with **limited impact on booking and cancellation rates**.

This suggests that behavioral rates are robust, but signal density per user is reduced.

###### Cohort deifinition: sessions > 3

In [ ]:
%sql
-- select sessions with session_start > '2023-01-04'
WITH filtered_sessions AS (
  SELECT
    user_id,
    trip_id,
    cancellation,
    flight_booked,
    hotel_booked
  FROM sessions
  WHERE session_start > '2023-01-04'
),

-- select eligible users from filtered_sessions
eligible_users AS (
  SELECT user_id
  FROM filtered_sessions
  GROUP BY user_id
  HAVING COUNT(*) > 3  -- users with more than 3 sessions after 2023-01-04
),

-- select cohort users in filtered_sessions
cohort_sessions AS (
  SELECT fs.*
  FROM filtered_sessions fs
  JOIN eligible_users eu ON eu.user_id = fs.user_id
),

-- select all sessions from cohort sessions
sessions_by_user AS (
  SELECT
    user_id,
    COUNT(*) AS total_sessions,
    COUNT(*) FILTER (WHERE trip_id IS NULL) AS browsing_sessions, -- only browsing sessions
    COUNT(*) FILTER (
      WHERE trip_id IS NOT NULL AND cancellation = FALSE
      AND (flight_booked = TRUE OR hotel_booked = TRUE) -- only booking sessions
    ) AS booking_sessions,
    COUNT(*) FILTER (WHERE cancellation = TRUE) AS cancellation_sessions -- only cancellation sessions
  FROM cohort_sessions
  GROUP BY user_id
),

/* select user by gender from filtered sessions_by_user
now to reduce the query execution time */
users_with_gender AS (
  SELECT
    sbu.*,
    u.gender
  FROM sessions_by_user sbu
  JOIN users u ON u.user_id = sbu.user_id
)

-- select all the information from users_with_gender and perform aggregations
SELECT
  gender,
  COUNT(*) AS n_users,
  SUM(browsing_sessions) AS n_browsings, -- total number of browsing sessions
  SUM(booking_sessions) AS n_bookings,   -- total number of booking sessions
  SUM(cancellation_sessions) AS n_cancels, -- total number of cancellations
  SUM(total_sessions) AS n_sessions, -- total number of sessions
  ROUND(SUM(booking_sessions)::numeric / NULLIF(SUM(total_sessions), 0), 3) AS booking_rate, -- booking rate
  ROUND(SUM(cancellation_sessions)::numeric / NULLIF(SUM(booking_sessions), 0), 3) AS cancel_rate -- cancellation rate
FROM users_with_gender
GROUP BY gender -- group by gender
ORDER BY n_users DESC;


gender,n_users,n_browsings,n_bookings,n_cancels,n_sessions,booking_rate,cancel_rate
F,203043,614589,382842,12058,1009489,0.379,0.031
M,173668,469290,319168,9692,798150,0.400,0.030
O,2571,6946,4758,157,11861,0.401,0.033


**Note**: A threshold of more than 3 sessions substantially increases the user base and yields a **more balanced demographic distribution**.  
At the same time, it introduces a large share of **low-engagement** users, inflating browsing activity without adding meaningful booking signal.

This cohort favors coverage over behavioral depth and is therefore less suitable for segmentation.

##### 5.4.3 Cohort selection rationale

**Conclusion**:
- High thresholds prioritize behavioral depth but reduce demographic balance.
- Low thresholds improve coverage but dilute individual-level signals.
- The selected threshold (>7 sessions) prioritizes behavioral depth while preserving stable booking and cancellation dynamics, making it suitable for segmentation.

The final cohort therefore includes users with more than **7 sessions after 2023-01-04**, providing sufficient behavioral history for downstream user-level aggregation and segmentation.

**Note**: Overall, booking and cancellation rates remain remarkably stable across cohort definitions, indicating that the selected threshold mainly affects user coverage and behavioral depth rather than conversion dynamics.

#### Key Findings

- The database structure is coherent and relationally well-defined, with clean primary keys and no duplicate identifiers across the four core tables.

- Three distinct session types emerge clearly from the data:
browsing-only sessions (`trip_id IS NULL`),
booking sessions (`trip_id NOT NULL AND cancellation = FALSE`),
cancellation sessions (`trip_id NOT NULL AND cancellation = TRUE`).

- Cancellation sessions systematically retain all booking-related flags set to TRUE.
Consequently, the original booking composition must be retrieved from the corresponding booking session when analyzing booking behavior.

- Each cancelled trip consistently consists of exactly two sessions: one booking session followed by one cancellation session.
This confirms a stable and interpretable trip lifecycle at the session level.

- Session duration and page-click distributions reveal a clear separation between short browsing interactions and longer, more engaged booking sessions, supporting their use as meaningful behavioral signals.

- User demographics appear realistic and well balanced, with a strong concentration of working-age travelers and a geographic skew toward North America.

- Flight and hotel activity is heavily concentrated in 2023, justifying the focus on a recent and temporally consistent analysis window aligned with the project objectives.

- Minor data quality issues (e.g. negative hotel nights) are present but isolated and are addressed explicitly in the subsequent data preparation notebook.

- Overall, the analysis confirms that a **semantically consistent and analytically reliable session-level dataset** can be constructed by integrating sessions with user, flight, and hotel data, provided that booking-cancellation temporal relationships are handled explicitly.

These findings provide a validated foundation for user-level aggregation and feature engineering in the subsequent notebooks.

### 6. Cancellations and Temporal Consistency

An important aspect emerging from the exploration of the sessions table is the presence of cancellation sessions whose associated booking sessions may fall outside the selected temporal window.

In particular, some trips are booked before the cohort start date but cancelled after it.
Excluding these booking sessions would result in incomplete trip life cycles and potentially inconsistent interpretations of user behavior.

For this reason, booking sessions associated with cancellations occurring within the cohort window are selectively reintroduced, ensuring a coherent session-level representation of each trip.

**Problem**  
Filtering sessions strictly by date may exclude booking events for trips that are cancelled later,
leading to incomplete or misleading session-level representations.

**Solution**  
Booking sessions associated with cancellations occurring within the cohort window are selectively reintroduced,
ensuring a complete and temporally consistent view of each trip lifecycle.

### 7. Save Session Level Dataset

##### Session-Level Query

Following the guidance from the Head of Marketing, the analysis focuses on users with:
- sessions occurring after 2023-01-04,
- more than 7 recorded sessions.

This cohort definition aims to ensure sufficient behavioral history per user while limiting bias introduced by very recent or inactive users.

Additionally, focusing on recent data reduces potential distortions related to historical effects such as the COVID-19 period.
Given the objectives of customer segmentation and perk allocation, this time-bounded snapshot provides a reasonable and business-aligned representation of active users.

While alternative cohort definitions were considered, this approach was deemed sufficiently robust
for the purpose of customer segmentation and reward design, balancing behavioral depth and recency.

In [ ]:
# ---------------------------------------------------------------
# Final session-level extraction query
#
# This query builds the consolidated session-level dataset used
# in the next notebooks. It applies:
#   - a recency filter (sessions after 2023-01-04),
#   - a minimum activity threshold (>7 sessions per user),
#   - and a temporal consistency rule that reintroduces
#     pre-cutoff booking sessions for trips cancelled later.
#
# The result is a coherent session-level dataset ready for
# cleaning and EDA in Notebook 03.
# ---------------------------------------------------------------

# Run the session-level query and create a Spark DataFrame
session_df = spark.sql("""
WITH
  -- 1. Sessions after cutoff data
  filtered_sessions AS (
    SELECT *
    FROM sessions
    WHERE session_start > '2023-01-04'
  ),

  -- 2. Users with sufficient activity
  filtered_users AS (
    SELECT
      user_id,
      COUNT(*) AS session_count
    FROM filtered_sessions
    GROUP BY user_id
    HAVING COUNT(*) > 7
  ),

  -- 3. Session-level table for filtered users
  session_level AS (
    SELECT
      -- SESSION FIELDS -------------
      s.session_id,
      s.user_id,
      s.trip_id,
      s.session_start,
      s.session_end,
      s.page_clicks,
      s.flight_discount,
      s.flight_discount_amount,
      s.flight_booked,
      s.hotel_discount,
      s.hotel_discount_amount,
      s.hotel_booked,
      s.cancellation,

      -- USERS FIELDS -------------
      u.birthdate,
      u.gender,
      u.married,
      u.has_children,
      u.home_country,
      u.home_city,
      u.home_airport,
      u.home_airport_lat,
      u.home_airport_lon,
      u.sign_up_date,

      -- FLIGHT FIELDS -------------
      f.origin_airport,
      f.destination,
      f.destination_airport,
      f.seats,
      f.return_flight_booked,
      f.departure_time,
      f.return_time,
      f.checked_bags,
      f.trip_airline,
      f.destination_airport_lat,
      f.destination_airport_lon,
      f.base_fare_usd,

      -- HOTEL FIELDS -------------
      h.hotel_name,
      h.nights,
      h.rooms,
      h.check_in_time,
      h.check_out_time,
      h.hotel_per_room_usd

    FROM filtered_sessions s
    INNER JOIN users u ON s.user_id = u.user_id
    LEFT JOIN flights f ON s.trip_id = f.trip_id
    LEFT JOIN hotels h ON s.trip_id = h.trip_id
    WHERE s.user_id IN (SELECT user_id FROM filtered_users)
  ),

  -- 4. Cancellation sessions inside the window
  cancellations AS (
    SELECT *
    FROM session_level
    WHERE cancellation = TRUE
  ),

  -- 5. Retrieve booking sessions BEFORE cutoff for trips cancelled AFTER cutoff
  sessions_trip_booked_before AS (
    SELECT *
    FROM sessions
    WHERE trip_id IN (
      SELECT trip_id
      FROM cancellations
    )
    AND session_start <= '2023-01-04'  -- booking occurred before the cutoff date
  ),

  -- 6.  Build session-level rows for these earlier booking sessions
  session_level_before AS (
    SELECT
      -- SESSION FIELDS -------------
      s.session_id,
      s.user_id,
      s.trip_id,
      s.session_start,
      s.session_end,
      s.page_clicks,
      s.flight_discount,
      s.flight_discount_amount,
      s.flight_booked,
      s.hotel_discount,
      s.hotel_discount_amount,
      s.hotel_booked,
      s.cancellation,

      -- USER FIELDS -------------
      u.birthdate,
      u.gender,
      u.married,
      u.has_children,
      u.home_country,
      u.home_city,
      u.home_airport,
      u.home_airport_lat,
      u.home_airport_lon,
      u.sign_up_date,

      -- FLIGHT FIELDS -------------
      f.origin_airport,
      f.destination,
      f.destination_airport,
      f.seats,
      f.return_flight_booked,
      f.departure_time,
      f.return_time,
      f.checked_bags,
      f.trip_airline,
      f.destination_airport_lat,
      f.destination_airport_lon,
      f.base_fare_usd,

      -- HOTEL FIELDS -------------
      h.hotel_name,
      h.nights,
      h.rooms,
      h.check_in_time,
      h.check_out_time,
      h.hotel_per_room_usd

    FROM sessions_trip_booked_before s
    INNER JOIN users u ON s.user_id = u.user_id
    LEFT JOIN flights f ON s.trip_id = f.trip_id
    LEFT JOIN hotels h ON s.trip_id = h.trip_id
    WHERE s.user_id IN (SELECT user_id FROM filtered_users)
  ),

  -- 7. Combine both parts
  final_tab AS (
    SELECT * FROM session_level
    UNION ALL
    SELECT * FROM session_level_before
  )

SELECT *
FROM final_tab;
""")

In [ ]:
# Convert Spark DataFrame to Pandas DataFrame
session_pdf = session_df.toPandas()

# Quick check of the first few rows
session_pdf.head(5)

,session_id,user_id,trip_id,session_start,session_end,page_clicks,flight_discount,flight_discount_amount,flight_booked,hotel_discount,hotel_discount_amount,hotel_booked,cancellation,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date,origin_airport,destination,destination_airport,seats,return_flight_booked,departure_time,return_time,checked_bags,trip_airline,destination_airport_lat,destination_airport_lon,base_fare_usd,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_per_room_usd
0,513347-53b8cae2741a47a38c3a0bd837fc6b06,513347,513347-a0ea4b8b2a6244da92a9bbb66a4e5af9,2023-01-29 18:24:00,2023-01-29 18:27:26,28,True,0.150000000000000000,True,False,None,True,False,1974-10-08,F,True,True,usa,los angeles,LAX,33.942000000000000000,-118.408000000000000000,2023-01-06,LAX,edmonton,YEG,1.0,True,2023-02-08 07:00:00,2023-02-11 07:00:00,1.0,Air Canada,53.667000000000000000,-113.467000000000000000,378.920000000000000000,Fairmont - edmonton,2.0,1.0,2023-02-08 11:47:03.930,2023-02-10 11:00:00,129.000000000000000000
1,514318-f6964894bb014d4190d4eaac6d49122a,514318,514318-48629aa17e58491b8605807a928a92ba,2023-01-29 11:24:00,2023-01-29 11:26:15,18,False,None,True,False,None,True,False,1971-07-27,F,True,True,usa,seattle,SEA,47.449000000000000000,-122.309000000000000000,2023-01-06,SEA,san diego,NZY,1.0,True,2023-02-06 09:00:00,2023-02-08 09:00:00,1.0,American Airlines,32.699000000000000000,-117.215000000000000000,311.310000000000000000,Choice Hotels - san diego,1.0,1.0,2023-02-06 13:07:09.570,2023-02-08 11:00:00,360.000000000000000000
2,544449-9ec11f30602645c587856aaf234f34bd,544449,544449-9c868af3e36048a09b8158d5e8e2e1cf,2023-01-29 19:50:00,2023-01-29 19:51:58,16,False,None,True,False,None,True,False,1979-06-17,M,True,True,usa,new york,LGA,40.777000000000000000,-73.872000000000000000,2023-01-21,LGA,houston,HOU,1.0,True,2023-02-08 10:00:00,2023-02-10 10:00:00,1.0,Southwest Airlines,29.607000000000000000,-95.159000000000000000,385.160000000000000000,Marriott - houston,1.0,1.0,2023-02-08 14:51:41.400,2023-02-10 11:00:00,151.000000000000000000
3,557508-c8e532ef18064eba946c78d2dfde7438,557508,557508-c5fe10a144384c99b8476b05b81ad958,2023-01-29 12:41:00,2023-01-29 12:43:58,24,False,None,True,True,0.050000000000000000,True,False,1989-06-26,F,False,False,usa,san jose,SJC,37.362000000000000000,-121.929000000000000000,2023-01-28,SJC,washington,IAD,2.0,True,2023-02-03 14:00:00,2023-02-06 14:00:00,1.0,American Airlines,38.852000000000000000,-77.037000000000000000,1457.320000000000000000,Rosewood - washington,1.0,2.0,2023-02-03 20:51:51.255,2023-02-05 11:00:00,102.000000000000000000
4,559863-83ccf47914b549889033016aa8d81e7b,559863,559863-8d830c0943044feeb019afe6e9d7a5fe,2023-01-29 21:54:00,2023-01-29 21:57:34,29,False,None,True,False,None,True,False,1990-05-05,F,False,False,usa,indianapolis,IND,39.717000000000000000,-86.294000000000000000,2023-01-29,IND,columbus,LCK,3.0,True,2023-02-05 10:00:00,2023-02-09 10:00:00,0.0,Allegiant Air,39.998000000000000000,-82.892000000000000000,152.180000000000000000,Crowne Plaza - columbus,3.0,3.0,2023-02-05 12:21:54.180,2023-02-09 11:00:00,64.000000000000000000


In [ ]:
# Quick check
session_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49218 entries, 0 to 49217
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   session_id               49218 non-null  object        
 1   user_id                  49218 non-null  int32         
 2   trip_id                  16709 non-null  object        
 3   session_start            49218 non-null  datetime64[ns]
 4   session_end              49218 non-null  datetime64[ns]
 5   page_clicks              49218 non-null  int32         
 6   flight_discount          49218 non-null  bool          
 7   flight_discount_amount   8282 non-null   object        
 8   flight_booked            49218 non-null  bool          
 9   hotel_discount           49218 non-null  bool          
 10  hotel_discount_amount    6206 non-null   object        
 11  hotel_booked             49218 non-null  bool          
 12  cancellation             49218 n

In [ ]:
# Export the DataFrame to CSV
# Save to CSV, overwriting if the file already exists
csv_path = "../data/raw/session_level_raw.csv"
session_pdf.to_csv(csv_path, index=False)

print(f"Session-level CSV saved to: {csv_path}")

Session-level CSV saved to: ../data/raw/session_level_raw.csv


The outcome of this notebook is a session-level dataset that:
- is temporally consistent,
- preserves key behavioral and demographic signals,
- accounts for booking and cancellation dynamics,
- is ready to be reused for further cleaning, aggregation, and exploratory analysis.

The resulting dataset is saved and reused in subsequent notebooks, where user-level aggregation and advanced EDA are performed.

### 8. Summary

- The TravelTide database structure and relationships were analyzed and validated.
- No critical data quality issues were identified, aside from expected NULLs in browsing-only sessions.
- A cohort definition aligned with business objectives was selected.
- A temporally consistent session-level dataset was constructed, accounting for cancellations.
- The resulting dataset is ready for cleaning, feature engineering, and aggregation in Notebook 03.